# Розширення генерації на основі пошуку (RAG) та векторні бази даних

In [ ]:
%pip install mistralai getenv openai faiss-cpu pandas numpy  

In [2]:
import os
import pandas as pd
import numpy as np
import faiss

## Створення нашої бази знань

Налаштування FAISS для векторного пошуку


In [ ]:
# Шляхи до файлів (даних для RAG)
data_paths = [
    "data/frameworks.md", 
    "data/own_framework.md", 
    "data/perceptron.md"
] 

# Ініціалізація порожнього DataFrame
df = pd.DataFrame(columns=['path', 'text'])

# Сучасний спосіб додавання рядків до DataFrame
for path in data_paths:
    try:
        with open(path, 'r', encoding='utf-8') as file:
            file_content = file.read()
        
        # Використовуємо concat замість застарілого append
        new_row = pd.DataFrame({'path': [path], 'text': [file_content]})
        df = pd.concat([df, new_row], ignore_index=True)
    except FileNotFoundError:
        print(f"Файл не знайдено: {path}")

df.head()

In [ ]:
def split_text(text, max_length, min_length):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(' '.join(current_chunk)) < max_length and len(' '.join(current_chunk)) > min_length:
            chunks.append(' '.join(current_chunk))
            current_chunk = []

    # Якщо останній фрагмент не досягнув мінімальної довжини, все одно додати його
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Припускаючи, що analyzed_df - це pandas DataFrame, а 'output_content' - це стовпець у цьому DataFrame
splitted_df = df.copy()
splitted_df['chunks'] = splitted_df['text'].apply(lambda x: split_text(x, 400, 300))

splitted_df

In [ ]:
# Припускаючи, що 'chunks' - це стовпець списків у DataFrame splitted_df, ми розділимо фрагменти на різні рядки
flattened_df = splitted_df.explode('chunks')

flattened_df.head()

## Перетворення нашого тексту на ембедінги

Перетворення нашого тексту на ембедінги із використанням сервісу [MistralAI](https:/docs.mistral.ai/capabilities/embeddings/) та зберігання їх для використання з FAISS

In [7]:
from mistralai import Mistral
from dotenv import load_dotenv

load_dotenv()

# Ініціалізація клієнта Mistral
api_key = os.getenv("MISTRAL_API_KEY")
assert api_key, "ERROR: MISTRAL_API_KEY is missing"

client_embedding = Mistral(api_key=api_key)

def create_embeddings(text, model="mistral-embed"):
    """
    Створює ембедінги для тексту, використовуючи Mistral AI.
    
    Args:
        text: Текст або список текстів для ембедінгу
        model: Модель для ембедінгу (за замовчуванням mistral-embed)
        
    Returns:
        Вектор ембедінгу
    """
    # Обробка pandas Series
    if isinstance(text, pd.Series):
        # Беремо перший елемент з Series
        text = text.iloc[0]
    
    # Перетворюємо в список рядків для API
    if not isinstance(text, list):
        text = [str(text)]
    else:
        text = [str(item) for item in text]
    
    # Виклик API Mistral для отримання ембедінгів
    embeddings_response = client_embedding.embeddings.create(
        model=model,
        inputs=text
    )
    
    # Повернення ембедінгу для першого елемента
    return embeddings_response.data[0].embedding

# Приклад використання:
embeddings = create_embeddings(flattened_df['chunks'][0])

In [ ]:
cat = create_embeddings("cat")
cat

In [ ]:
import pickle
from pathlib import Path

def save_embeddings(df, folder="embeddings", filename="flattened_df.pkl"):
    """
    Зберігає DataFrame з ембедінгами в указану папку.
    
    Args:
        df: DataFrame з ембедінгами
        folder: Назва папки для збереження
        filename: Ім'я файлу для збереження
    """
    # Створення директорії, якщо вона не існує
    Path(folder).mkdir(parents=True, exist_ok=True)
    
    # Шлях до файлу
    file_path = os.path.join(folder, filename)
    
    # Збереження DataFrame
    with open(file_path, 'wb') as f:
        pickle.dump(df, f)
    
    print(f"DataFrame успішно збережено в {file_path}")

def load_embeddings(folder="embeddings", filename="flattened_df.pkl"):
    """
    Завантажує DataFrame з ембедінгами з указаної папки.
    
    Args:
        folder: Назва папки для завантаження
        filename: Ім'я файлу для завантаження
        
    Returns:
        DataFrame з ембедінгами або None, якщо файл не існує
    """
    # Шлях до файлу
    file_path = os.path.join(folder, filename)
    
    # Перевірка існування файлу
    if os.path.exists(file_path):
        # Завантаження DataFrame
        with open(file_path, 'rb') as f:
            df = pickle.load(f)
        
        print(f"DataFrame успішно завантажено з {file_path}")
        return df
    else:
        print(f"Файл {file_path} не знайдено")
        return None

def get_or_create_embeddings(df, chunk_column, embedding_function, folder="embeddings", filename="flattened_df.pkl"):
    """
    Завантажує DataFrame з ембедінгами або створює новий.
    
    Args:
        df: Вихідний DataFrame з текстами
        chunk_column: Назва стовпця з текстовими фрагментами
        embedding_function: Функція для створення ембедінгів
        folder: Назва папки для збереження/завантаження
        filename: Ім'я файлу для збереження/завантаження
        
    Returns:
        DataFrame з ембедінгами
    """
    # Спроба завантажити DataFrame
    loaded_df = load_embeddings(folder, filename)
    
    if loaded_df is not None:
        return loaded_df
    
    # Якщо завантаження не вдалося, створюємо ембедінги
    print("Створення нових ембедінгів...")
    
    # Створення ембедінгів
    embeddings = []
    for chunk in df[chunk_column]:
        embeddings.append(embedding_function(chunk))
    
    # Збереження ембедінгів в DataFrame
    df['embeddings'] = embeddings
    
    # Збереження DataFrame
    save_embeddings(df, folder, filename)
    
    return df


# Використовуємо функцію, яка розраховує ембедінги, 
# якщо вони не були раніше створені і збережені в папці в "15-rag-and-vector-databases/embeddings"
flattened_df = get_or_create_embeddings(
    splitted_df.explode('chunks'), 
    'chunks', 
    create_embeddings
)

flattened_df.head()

# Пошук з використанням FAISS

Векторний пошук та схожість між нашим запитом і базою даних з використанням FAISS

### Створення індексу FAISS та підготовка до пошуку

In [ ]:
# Отримуємо ембедінги як масив numpy
embeddings_list = flattened_df['embeddings'].to_list()
embeddings_array = np.array(embeddings_list).astype('float32')

# Визначаємо розмірність векторів
vector_dimension = len(embeddings_list[0])

# Створюємо індекс FAISS
index = faiss.IndexFlatL2(vector_dimension)  # L2 - це евклідова відстань

# Додаємо наші вектори до індексу
index.add(embeddings_array)

# Перевіряємо кількість векторів в індексі
print(f"Загальна кількість векторів в індексі: {index.ntotal}, розмірність векторів: {vector_dimension}")

In [ ]:
# Ваше текстове запитання
question = "Що таке персептрон?"

# Перетворіть запитання у вектор запиту
query_vector = create_embeddings(question)  
query_vector_array = np.array([query_vector]).astype('float32')

# Знайдіть найбільш схожі документи (k=5 - скільки найближчих сусідів шукаємо)
k = 5
distances, indices = index.search(query_vector_array, k)

# Виведіть найбільш схожі документи
for i in range(min(3, len(indices[0]))):
    idx = indices[0][i]
    print(f"Фрагмент {i+1}:")
    print(flattened_df['chunks'].iloc[idx])
    print(f"Шлях: {flattened_df['path'].iloc[idx]}")
    print(f"Відстань: {distances[0][i]}")
    print("-" * 50)

## Поєднання всього для відповіді на запитання

In [12]:
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import ChatCompletionsToolDefinition
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

# Виберіть модель загального призначення для тексту
deployment = "gpt-4o-mini"

# Реалізація чатботів (при наявності і відсутності RAG)

In [13]:
def chatbot_with_rag(user_input):
    # Перетворіть запитання у вектор запиту
    query_vector = create_embeddings(user_input)
    query_vector_array = np.array([query_vector]).astype('float32')
    
    # Знайдіть найбільш схожі документи з FAISS
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)

    # додайте документи до запиту, щоб забезпечити контекст
    history = []
    for idx in indices[0]:
        history.append(flattened_df['chunks'].iloc[idx])

    # створюємо об'єкт повідомлення з контекстом
    context = "\n\n".join(history)  # всі знайдені фрагменти

    # Формуємо запит, що просить коротку, але завершену відповідь
    messages = [
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. "},
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion: {user_input}\n\n Provide a brief but complete answer based on the context. Answer in Ukrainian."}
    ]


    response = client.complete(
        temperature=0,
        model=deployment,
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0].message.content


def chatbot_without_rag(user_input):
    """
    Чат-бот без використання RAG (прямий запит до моделі).
    """
    messages=[
        {"role": "system", "content": "You are an AI assistant that helps with AI questions. Provide brief but complete answers. Answer in Ukrainian."},
        {"role": "user", "content": f"Question: {user_input}"}
    ]

    response = client.complete(
        temperature=0,
        model=deployment,
        messages=messages,
        max_tokens=300,
    )

    return response.choices[0].message.content

# Порівняння відповідей RAG-системи

In [14]:
from IPython.display import display, Markdown, HTML

def compare_responses(user_input, save_to_file=False, filename="rag_comparison.md"):
    """
    Порівнює відповіді чат-боту з RAG та без RAG.
    
    Args:
        user_input: Запитання користувача
        save_to_file: Зберегти результат у файл Markdown
        filename: Назва файлу для збереження
    """
    # Отримання знайдених чанків
    query_vector = create_embeddings(user_input)
    query_vector_array = np.array([query_vector]).astype('float32')
    k = 5  # кількість найближчих сусідів для пошуку
    distances, indices = index.search(query_vector_array, k)
    
    # Отримання відповідей
    rag_response = chatbot_with_rag(user_input)
    no_rag_response = chatbot_without_rag(user_input)
    
    # Формування markdown-тексту
    markdown_text = f"""
# Порівняння відповідей

## 📝 Запит: {user_input}

## 🔍 Відповіді моделей

### Без використання RAG

{no_rag_response}

### З використанням RAG

{rag_response}

## 📚 Знайдені фрагменти тексту

"""
    
    # Додавання чанків
    for i, idx in enumerate(indices[0]):
        chunk_content = flattened_df['chunks'].iloc[idx]
        path = flattened_df['path'].iloc[idx]
        dist = float(distances[0][i])
        
        markdown_text += f"""
### Фрагмент {i+1} (відстань: {dist:.4f})

**Шлях**: {path}

{chunk_content}

"""
    
    # Виведення Markdown
    display(Markdown(markdown_text))
    
    # Для коректного відображення формул
    mathjax_script = """
    <script type="text/javascript">
        MathJax = {
            tex: {
                inlineMath: [['$', '$']]
            }
        };
    </script>
    <script type="text/javascript" id="MathJax-script" async
        src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-chtml.js">
    </script>
    """
    display(HTML(mathjax_script))
    
    # Збереження в файл, якщо потрібно
    if save_to_file:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(markdown_text)
        print(f"Результати збережено у файл: {filename}")


In [ ]:
# Приклад використання:
compare_responses("Що таке перцептрон?")